<a href="https://colab.research.google.com/github/MLJCUnito/ProjectX2020/blob/master/HowToTackleAMLCompetition/Lecture7%7CAdvancedFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-bio geopandas libpysal

     |████████████████████████████████| 10.5MB 3.3MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.1 which is incompatible.
  Found existing installation: pandas 0.20.0
    Uninstalling pandas-0.20.0:
      Successfully uninstalled pandas-0.20.0


http://readiab.org/ Applied Bioinformatics in Python

### K-Nearest-Neighbors (KNN)

In [ ]:
!wget https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_municipalities.geojson

--2020-08-27 12:57:32--  https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_municipalities.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37928373 (36M) [text/plain]
Saving to: ‘limits_IT_municipalities.geojson.1’

limits_IT_municipal 100%[===================>]  36.17M  63.7MB/s    in 0.6s    

2020-08-27 12:57:33 (63.7 MB/s) - ‘limits_IT_municipalities.geojson.1’ saved [37928373/37928373]



In [ ]:
import geopandas as gpd 
import matplotlib.pyplot as plt 
import libpysal.weights as lp 

ImportError: ignored

In [ ]:
municip = gpd.read_file("limits_IT_municipalities.geojson")
piemonte_municip = municip[municip.reg_name == "Piemonte"]
torino_prov = piemonte_municip[piemonte_municip.prov_name == "Torino"]
torino_prov.head()

In [ ]:
torino_prov.plot()

In [ ]:
torino_prov.shape

In [ ]:
w_queen = lp.Queen.from_dataframe(torino_prov)

In [ ]:
w_queen.plot(torino_prov)

### A Non-Euclidean Distance: Bray-Curtis

Definition, a few lines on non-euclidean distances

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

sample_ids = ['A', 'B', 'C']
feature_ids = ['OTU1', 'OTU2', 'OTU3', 'OTU4', 'OTU5']
data = np.array([[1, 0, 0],
                 [3, 2, 0],
                 [0, 0, 6],
                 [1, 4, 2],
                 [0, 4, 1]])

table1 = pd.DataFrame(data, index=feature_ids, columns=sample_ids)
table1

In [ ]:
#from scipy.spatial.distance import braycurtis
from skbio.stats.distance import DistanceMatrix

In [ ]:
def bray_curtis_distance(table, sample1_id, sample2_id):
    numerator = 0
    denominator = 0
    sample1_counts = table[sample1_id]
    sample2_counts = table[sample2_id]
    for sample1_count, sample2_count in zip(sample1_counts, sample2_counts):
        numerator += abs(sample1_count - sample2_count)
        denominator += sample1_count + sample2_count
    return numerator / denominator

In [ ]:
def df_to_distances(table, pairwise_distance_fn):
    sample_ids = table.columns
    num_samples = len(sample_ids)
    data = np.zeros((num_samples, num_samples))
    for i, sample1_id in enumerate(sample_ids):
        for j, sample2_id in enumerate(sample_ids[:i]):
            data[i,j] = data[j,i] = pairwise_distance_fn(table, sample1_id, sample2_id)
    return DistanceMatrix(data, sample_ids)

In [ ]:
df_to_distances(table1, bray_curtis_distance)